**Import Libraries**

In [30]:
import pandas as pd
import numpy as np
import pickle
import gc
from sklearn.preprocessing import LabelEncoder
import warnings
warnings.filterwarnings('ignore')

**MOUNT GOOGLE DRIVE**

In [31]:
from google.colab import drive
drive.mount('/content/drive')

import os
project_folder = '/content/drive/MyDrive/FraudDetection'  # Sesuaikan dengan folder kamu
os.chdir(project_folder)

print(f"Working Directory: {os.getcwd()}")
print("\n✅ Google Drive mounted!")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Working Directory: /content/drive/MyDrive/FraudDetection

✅ Google Drive mounted!


**LOAD DATA & COLUMN INFO**

In [32]:
print("\n" + "=" * 60)
print("LOADING CLEANED DATA")
print("=" * 60)

# Load cleaned data from Notebook 2
train_df = pd.read_csv('data/cleaned/train_cleaned.csv')
print(f"Dataset Shape: {train_df.shape}")
print(f"Memory Usage: {train_df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

# Load column info
with open('results/02_updated_column_info.pkl', 'rb') as f:
    column_info = pickle.load(f)

numeric_cols = column_info['numeric_columns']
categorical_cols = column_info['categorical_columns']

print(f"\nNumeric Columns: {len(numeric_cols)}")
print(f"Categorical Columns: {len(categorical_cols)}")


LOADING CLEANED DATA
Dataset Shape: (590540, 220)
Memory Usage: 1214.67 MB

Numeric Columns: 210
Categorical Columns: 9


**SEPARATE TARGET VARIABLE**

In [33]:
print("\n" + "=" * 60)
print("SEPARATING FEATURES AND TARGET")
print("=" * 60)

# Check if TransactionID exists and store it
if 'TransactionID' in train_df.columns:
    transaction_ids = train_df['TransactionID'].copy()
    train_df = train_df.drop('TransactionID', axis=1)
    print("✓ Stored TransactionID for later use")

    # Remove from column lists
    if 'TransactionID' in numeric_cols:
        numeric_cols.remove('TransactionID')
    if 'TransactionID' in categorical_cols:
        categorical_cols.remove('TransactionID')

# Separate target
y = train_df['isFraud'].copy()
X = train_df.drop('isFraud', axis=1)

print(f"\nFeatures (X): {X.shape}")
print(f"Target (y): {y.shape}")
print(f"\nTarget Distribution:")
print(y.value_counts())


SEPARATING FEATURES AND TARGET
✓ Stored TransactionID for later use

Features (X): (590540, 218)
Target (y): (590540,)

Target Distribution:
isFraud
0    569877
1     20663
Name: count, dtype: int64


**CREATE NEW FEATURES (IF APPLICABLE)**

In [34]:
print("\n" + "=" * 60)
print("FEATURE ENGINEERING - CREATING NEW FEATURES")
print("=" * 60)

new_features_created = []

# Example: If there's a timestamp column, extract time-based features
# Adjust based on your actual column names
time_cols = [col for col in X.columns if 'time' in col.lower() or 'date' in col.lower()]

if len(time_cols) > 0:
    print(f"\nFound {len(time_cols)} time-related columns: {time_cols[:5]}")

    # Example feature engineering (uncomment and adjust as needed)
    # for col in time_cols[:1]:  # Process first time column as example
    #     if X[col].dtype in ['int64', 'float64']:
    #         # Create hour, day features if it's a timestamp
    #         X[f'{col}_hour'] = (X[col] // 3600) % 24
    #         X[f'{col}_day'] = (X[col] // 86400) % 7
    #         new_features_created.extend([f'{col}_hour', f'{col}_day'])
    #         numeric_cols.extend([f'{col}_hour', f'{col}_day'])

    print(f"✓ Time-based features can be engineered here if needed")
else:
    print("✓ No obvious time columns found")

# Example: Amount-based features (if applicable)
amount_cols = [col for col in numeric_cols if 'amount' in col.lower() or 'transactionamt' in col.lower()]

if len(amount_cols) > 0:
    print(f"\nFound {len(amount_cols)} amount-related columns")
    # You can create log transforms, bins, etc.
    # for col in amount_cols[:1]:
    #     X[f'{col}_log'] = np.log1p(X[col])
    #     new_features_created.append(f'{col}_log')
    #     numeric_cols.append(f'{col}_log')
    print(f"✓ Amount-based features can be engineered here if needed")

print(f"\nNew features created: {len(new_features_created)}")
if len(new_features_created) > 0:
    print(f"Feature names: {new_features_created}")


FEATURE ENGINEERING - CREATING NEW FEATURES
✓ No obvious time columns found

Found 1 amount-related columns
✓ Amount-based features can be engineered here if needed

New features created: 0


**ENCODE CATEGORICAL VARIABLES**

In [35]:
print("\n" + "=" * 60)
print("ENCODING CATEGORICAL VARIABLES")
print("=" * 60)

# Filter categorical columns that still exist in X
categorical_cols = [col for col in categorical_cols if col in X.columns]

print(f"\nCategorical columns to encode: {len(categorical_cols)}")

label_encoders = {}
encoding_mappings = {}

if len(categorical_cols) > 0:
    print("\nEncoding categorical columns with Label Encoding...")

    for i, col in enumerate(categorical_cols, 1):
        if i % 10 == 0:
            print(f"  Processed {i}/{len(categorical_cols)} columns...")

        # Convert to string type first
        X[col] = X[col].astype(str)

        # Initialize and fit label encoder
        le = LabelEncoder()
        X[col] = le.fit_transform(X[col])

        # Store encoder and mapping
        label_encoders[col] = le
        encoding_mappings[col] = dict(zip(le.classes_, le.transform(le.classes_)))

    print(f"\n✓ Encoded {len(categorical_cols)} categorical columns")

    # Show sample encoding
    sample_col = categorical_cols[0]
    print(f"\nSample encoding for '{sample_col}':")
    sample_mapping = list(encoding_mappings[sample_col].items())[:5]
    for original, encoded in sample_mapping:
        print(f"  '{original}' → {encoded}")
    if len(encoding_mappings[sample_col]) > 5:
        print(f"  ... and {len(encoding_mappings[sample_col]) - 5} more mappings")
else:
    print("✓ No categorical columns to encode")

# Save encoders for test data
with open('results/03_label_encoders.pkl', 'wb') as f:
    pickle.dump(label_encoders, f)

print("\n💾 Saved: results/03_label_encoders.pkl")

# Save encoding mappings for reference
with open('results/03_encoding_mappings.pkl', 'wb') as f:
    pickle.dump(encoding_mappings, f)

print("💾 Saved: results/03_encoding_mappings.pkl")


ENCODING CATEGORICAL VARIABLES

Categorical columns to encode: 9

Encoding categorical columns with Label Encoding...

✓ Encoded 9 categorical columns

Sample encoding for 'ProductCD':
  'C' → 0
  'H' → 1
  'R' → 2
  'S' → 3
  'W' → 4

💾 Saved: results/03_label_encoders.pkl
💾 Saved: results/03_encoding_mappings.pkl


**FEATURE SELECTION (CORRELATION-BASED)**

In [36]:
print("\n" + "=" * 60)
print("FEATURE SELECTION - REMOVING HIGH CORRELATION")
print("=" * 60)

# Calculate correlation matrix (for numeric features only)
print("\nCalculating correlation matrix...")
correlation_matrix = X.corr().abs()

# Find features with correlation > 0.95
upper_triangle = correlation_matrix.where(
    np.triu(np.ones(correlation_matrix.shape), k=1).astype(bool)
)

high_corr_features = [column for column in upper_triangle.columns
                      if any(upper_triangle[column] > 0.95)]

print(f"\nFeatures with correlation > 0.95: {len(high_corr_features)}")

if len(high_corr_features) > 0:
    print(f"\nRemoving {len(high_corr_features)} highly correlated features...")
    for feat in high_corr_features[:10]:  # Show first 10
        print(f"  - {feat}")
    if len(high_corr_features) > 10:
        print(f"  ... and {len(high_corr_features) - 10} more")

    X = X.drop(columns=high_corr_features)

    # Update column lists
    numeric_cols = [col for col in numeric_cols if col not in high_corr_features]

    print(f"\n✓ Removed highly correlated features")
else:
    print("✓ No highly correlated features found (threshold: 0.95)")

print(f"\nFinal feature count: {X.shape[1]}")

# Save list of removed features
with open('results/03_removed_corr_features.pkl', 'wb') as f:
    pickle.dump(high_corr_features, f)


FEATURE SELECTION - REMOVING HIGH CORRELATION

Calculating correlation matrix...

Features with correlation > 0.95: 55

Removing 55 highly correlated features...
  - C4
  - C6
  - C8
  - C10
  - C11
  - C12
  - C14
  - V11
  - V16
  - V18
  ... and 45 more

✓ Removed highly correlated features

Final feature count: 163


**VERIFY DATA QUALITY**

In [37]:
print("\n" + "=" * 60)
print("DATA QUALITY VERIFICATION")
print("=" * 60)

print("\nChecking for issues...")

# Check for missing values
missing_count = X.isnull().sum().sum()
print(f"✓ Missing values: {missing_count}")

# Check for infinite values
inf_count = np.isinf(X.select_dtypes(include=[np.number])).sum().sum()
print(f"✓ Infinite values: {inf_count}")

# Check data types
print(f"✓ All numeric: {X.select_dtypes(include=[np.number]).shape[1] == X.shape[1]}")

# Check shape consistency
print(f"✓ X shape: {X.shape}")
print(f"✓ y shape: {y.shape}")
print(f"✓ Shapes match: {X.shape[0] == y.shape[0]}")

if missing_count > 0 or inf_count > 0:
    print("\n⚠️ Warning: Data quality issues detected!")
else:
    print("\n✅ Data quality verified - Ready for model training!")


DATA QUALITY VERIFICATION

Checking for issues...
✓ Missing values: 0
✓ Infinite values: 0
✓ All numeric: True
✓ X shape: (590540, 163)
✓ y shape: (590540,)
✓ Shapes match: True

✅ Data quality verified - Ready for model training!


**SAVE ENGINEERED FEATURES**

In [39]:
print("\n" + "=" * 60)
print("SAVING ENGINEERED FEATURES")
print("=" * 60)

# Save features and target
X.to_csv('data/processed/X_features.csv', index=False)
y.to_csv('data/processed/y_target.csv', index=False, header=True)

print(f"💾 Saved: data/processed/X_features.csv")
print(f"   Shape: {X.shape}")
print(f"   Size: {os.path.getsize('data/processed/X_features.csv') / 1024**2:.2f} MB")

print(f"💾 Saved: data/processed/y_target.csv")
print(f"   Shape: {y.shape}")

# Save TransactionID if exists
if 'transaction_ids' in locals():
    transaction_ids.to_csv('data/processed/transaction_ids.csv', index=False, header=True)
    print(f"💾 Saved: data/processed/transaction_ids.csv")

# Save final column info
final_column_info = {
    'feature_columns': X.columns.tolist(),
    'target_column': 'isFraud',
    'numeric_columns': [col for col in numeric_cols if col in X.columns],
    'encoded_categorical_columns': list(label_encoders.keys()),
    'new_features': new_features_created,
    'removed_features': high_corr_features,
    'total_features': X.shape[1]
}

with open('results/03_final_column_info.pkl', 'wb') as f:
    pickle.dump(final_column_info, f)

print("💾 Saved: results/03_final_column_info.pkl")


SAVING ENGINEERED FEATURES
💾 Saved: data/processed/X_features.csv
   Shape: (590540, 163)
   Size: 373.41 MB
💾 Saved: data/processed/y_target.csv
   Shape: (590540,)
💾 Saved: data/processed/transaction_ids.csv
💾 Saved: results/03_final_column_info.pkl


**FEATURE ENGINEERING SUMMARY**

In [38]:
print("\n" + "=" * 60)
print("FEATURE ENGINEERING SUMMARY")
print("=" * 60)

summary = f"""
FEATURE ENGINEERING COMPLETED SUCCESSFULLY!

Input Shape: {train_df.shape}
Output Shape: X={X.shape}, y={y.shape}

Steps Completed:
1. ✓ Separated features and target variable
2. ✓ Created {len(new_features_created)} new features
3. ✓ Encoded {len(categorical_cols)} categorical variables using Label Encoding
4. ✓ Removed {len(high_corr_features)} highly correlated features (>0.95)
5. ✓ Verified data quality (no missing/infinite values)

Final Dataset:
- Total features: {X.shape[1]}
- Numeric features: {len([col for col in numeric_cols if col in X.columns])}
- Encoded categorical: {len(label_encoders)}
- Target distribution: {dict(y.value_counts())}
- All features are numeric: ✓
- Ready for scaling and modeling: ✓

Files Saved:
- X_features.csv: Feature matrix
- y_target.csv: Target variable
- label_encoders.pkl: Encoders for test data
- encoding_mappings.pkl: Encoding reference
- final_column_info.pkl: Feature metadata

Next Step: Proceed to Notebook 4 for Handling Imbalance & Scaling
"""

print(summary)

# Save summary
with open('results/03_feature_engineering_summary.txt', 'w') as f:
    f.write(summary)

print("💾 Saved: results/03_feature_engineering_summary.txt")


FEATURE ENGINEERING SUMMARY

FEATURE ENGINEERING COMPLETED SUCCESSFULLY!

Input Shape: (590540, 219)
Output Shape: X=(590540, 163), y=(590540,)

Steps Completed:
1. ✓ Separated features and target variable
2. ✓ Created 0 new features
3. ✓ Encoded 9 categorical variables using Label Encoding
4. ✓ Removed 55 highly correlated features (>0.95)
5. ✓ Verified data quality (no missing/infinite values)

Final Dataset:
- Total features: 163
- Numeric features: 154
- Encoded categorical: 9
- Target distribution: {0: np.int64(569877), 1: np.int64(20663)}
- All features are numeric: ✓
- Ready for scaling and modeling: ✓

Files Saved:
- X_features.csv: Feature matrix
- y_target.csv: Target variable
- label_encoders.pkl: Encoders for test data
- encoding_mappings.pkl: Encoding reference
- final_column_info.pkl: Feature metadata

Next Step: Proceed to Notebook 4 for Handling Imbalance & Scaling

💾 Saved: results/03_feature_engineering_summary.txt


**MEMORY CLEANUP**

In [40]:
print("\n" + "=" * 60)
print("MEMORY CLEANUP")
print("=" * 60)

del train_df, column_info, correlation_matrix, upper_triangle
gc.collect()

print("✓ Memory cleaned")
print("\n" + "=" * 60)
print("NOTEBOOK 3 COMPLETE!")
print("=" * 60)


MEMORY CLEANUP
✓ Memory cleaned

NOTEBOOK 3 COMPLETE!
